# UK Income Tax and National Insurance Reforms 2025

This notebook generates charts for the blog post analyzing income tax and NI reform options.

In [ ]:
from policyengine_uk import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.express as px
from policyengine_core.charts import format_fig

## Define reforms

In [ ]:
# NI reduction: 8% to 6%
NI_REDUCTION = {
    "gov.hmrc.national_insurance.rates.employee.main": {
        "2026-01-01.2030-12-31": 0.06,
    },
}

# Income tax increase: basic rate 20% -> 22%, higher rate 40% -> 42%
INCOME_TAX_INCREASE = {
    "gov.hmrc.income_tax.rates.uk[0].rate": {
        "2026-01-01.2030-12-31": 0.22,
    },
    "gov.hmrc.income_tax.rates.uk[1].rate": {
        "2026-01-01.2030-12-31": 0.42,
    },
}

# Threshold freeze extension to 2029-30
# (Would need actual threshold values here)
THRESHOLD_FREEZE = {
    # Add threshold freeze parameters
}

# Combined reform
COMBINED_REFORM = {}
COMBINED_REFORM.update(NI_REDUCTION)
COMBINED_REFORM.update(INCOME_TAX_INCREASE)

## Generate combined reform decile chart with animation by year

In [ ]:
baseline = Microsimulation()

reform = Reform.from_dict(COMBINED_REFORM)
reformed = Microsimulation(reform=reform)

# Generate data for all years
years = [2026, 2027, 2028, 2029]
df_year = []
df_decile = []
df_change = []

for year in years:
    decile = baseline.calculate("household_income_decile", year).clip(1, 10)
    
    baseline_income = baseline.calculate("household_net_income", year)
    reformed_income = reformed.calculate("household_net_income", year)
    
    # Calculate relative change by decile
    for d in range(1, 11):
        mask = decile == d
        baseline_decile_income = baseline_income[mask].sum()
        reformed_decile_income = reformed_income[mask].sum()
        relative_change = (reformed_decile_income - baseline_decile_income) / baseline_decile_income
        
        df_year.append(f"{year}-{year+1}")
        df_decile.append(d)
        df_change.append(relative_change * 100)

df = pd.DataFrame({
    "Year": df_year,
    "Decile": df_decile,
    "Change (%)": df_change,
})

# Create animated chart
fig = px.bar(
    df,
    x="Decile",
    y="Change (%)",
    animation_frame="Year",
    color="Change (%)",
    color_continuous_scale=["red", "white", "green"],
    color_continuous_midpoint=0,
    range_y=[-2, 1],
).update_layout(
    xaxis_title="Income decile",
    yaxis_title="Relative change in household income (%)",
    title="Combined reform: Change in household income by decile",
    xaxis_tickvals=list(range(1, 11)),
)

format_fig(fig)
fig.show()

## Export chart JSON for markdown

In [ ]:
import json

# Export to JSON for embedding in markdown
chart_json = fig.to_json()
print(chart_json)